# HTM - Preparación

## Carga de datos

Cargamos librerías y datos:

Las típicas (pandas, matplotlib, numpy)...

Funciones de sklearn de preprocesado y métricas.

Modelos y métricas de PyOD.

In [1]:
import pandas as pd
import numpy as np
from time import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import cohen_kappa_score

from pyod.models.mad import MAD

random_state = np.random.RandomState(42)

In [2]:
df = pd.read_csv('../../../Datasets/Dataset_2.csv',index_col='fecha', usecols=['fecha','FormacionNIRHumedadPV', 'Etapa2MWHumedadPV','Negro', 'CurvaCola', 'Congelado', 'Hum'])
df['Diferencia'] = df['FormacionNIRHumedadPV'] - df['Etapa2MWHumedadPV']
df = df.drop(['FormacionNIRHumedadPV','Etapa2MWHumedadPV'], axis=1)
df = df.loc[:,['Diferencia','Negro', 'CurvaCola', 'Congelado', 'Hum']]
df.index = pd.to_datetime(df.index)
df.head(2)

,Diferencia,Negro,CurvaCola,Congelado,Hum
fecha,,,,,
2021-02-10 10:00:00,-13.141570,1,0,1,0
2021-02-10 10:01:00,-12.412745,1,0,1,0


Unimos todas las anomalías en una columna, para posteriormente realizar las métricas tanto para todas las anomalías como para cada una en concreto.

Para ello, sumo las columnas de anomalías, y después las que son mayores que cero las establezco como uno (porque significa que alguna de las columnas si tenía anomalía registrada), las demás como cero.

In [3]:
df = df.drop(df[df['CurvaCola']==1].index)
df = df.drop('CurvaCola', axis=1)
df['Anomalia'] = df['Negro'] + df['Congelado'] + df['Hum']
df['Anomalia'] = df['Anomalia'].map(lambda x: 1 if x!=0 else 0)

lista_anomalias = ['Negro', 'Congelado', 'Hum', 'Anomalia']

atributos = df.columns.drop(lista_anomalias)
len(atributos)

1

In [4]:
df.to_csv('Datos/Entrada_limpia.csv')

In [5]:
#df = df['FormacionNIRHumedadPV']
df = df.iloc[40000:60000,]
df = df.loc[:,'Diferencia']
df

fecha
2021-03-23 16:05:00   -6.047450
2021-03-23 16:06:00   -5.487450
2021-03-23 16:07:00   -4.687060
2021-03-23 16:08:00   -3.715685
2021-03-23 16:09:00   -4.796080
                         ...   
2021-04-16 09:45:00   -4.662550
2021-04-16 09:46:00   -4.894310
2021-04-16 09:47:00   -5.063330
2021-04-16 09:48:00   -5.507845
2021-04-16 09:49:00   -5.030000
Name: Diferencia, Length: 20000, dtype: float64

In [6]:
df.to_csv('Datos/Entrada.csv', header=None)

In [7]:
filename = "Datos/Entrada.csv"

aux = open(filename,"r")
originaltext = aux.read()
aux.close()

aux = open(filename,"w")
aux.write("fecha,PVvsPred \ndatetime,float \nT, \n")
aux.write(originaltext)
aux.close()